In [248]:
import numpy as np

In [249]:
class Node:
    def __init__(self, position, adjacency):
        self.position = position
        self.adjacency = adjacency
    
    def __str__(self):
        return f"{self.position} {self.adjacency}"

In [250]:
def squareAdjacencyList(x1,x2,y1,y2): # constructs a non-periodic adjacency graph on a square lattice from (x1,y1) to (x2,y2)
    nodes = []
    
    for i in range(x1,x2+1):
        for j in range(y1,y2+1):
            adj = []
            if (i+1 <= x2):
                adj.append([i+1,j])
            if (i-1 >= x1):
                adj.append([i-1,j])
            if (j+1 <= y2):
                adj.append([i,j+1])
            if (j-1 >= y1):
                adj.append([i,j-1])
            nodes.append(Node([[i,j]],adj))

    return nodes

In [251]:
def solveLinearSystem(v1,v2,b): # returns x,y satisfying alpha*v1+beta*v2=b assuming v1 and v2 are linear independent
    A = np.transpose(np.array([v1,v2]))
    return np.dot(np.linalg.inv(A),b)

def coeffMatrix(v1,v2): # Z^2/([x,y]-[a,b],[x,y]-[c,d])
    a,b = v1
    c,d = v2

    # bounding box
    x1 = min(a,c,0) 
    x2 = max(a,c,0)
    y1 = min(b,d,0)
    y2 = max(b,d,0)
    nodes = squareAdjacencyList(x1,x2,y1,y2)

    # merge degenerate nodes
    for i in range(0,len(nodes)-1):
        for j in range(i+1,len(nodes)):
            a,b = nodes[i].position[0]
            c,d = nodes[j].position[0]
            sol = solveLinearSystem(v1,v2,[a-c,b-d])
            if (sol[0].is_integer() and sol[1].is_integer() and (sol[0] != 0 or sol[1] != 0)):
                for p in nodes[i].adjacency: # merge adjacencies
                    if p not in nodes[j].adjacency:
                        nodes[j].adjacency.append(p)
                for q in nodes[i].position: # merge positions
                    if q not in nodes[j].position:
                        nodes[j].position.append(q)
                nodes[i].adjacency = []


    mergedNodes = []
    # copy node list
    for node in nodes:
        if len(node.adjacency) != 0:
            mergedNodes.append(node) 

    N = len(mergedNodes)
    J = [[0 for col in range(N)] for row in range(N)]

    for i in range(0,N-1):
        for j in range(i+1,N):
            flag = False
            for r in mergedNodes[i].adjacency:
                if r in mergedNodes[j].position:
                    flag = True
            if flag:
                J[i][j] = 1
                J[j][i] = 1
        
    return np.array(J)

In [252]:
v1 = [-2,2] # first basis vector
v2 = [2,2] # second basis vector
J = coeffMatrix(v1,v2)
print(J)

[[0 1 1 0 1 0 1 0]
 [1 0 0 1 0 1 0 1]
 [1 0 0 1 0 1 0 1]
 [0 1 1 0 1 0 1 0]
 [1 0 0 1 0 1 0 1]
 [0 1 1 0 1 0 1 0]
 [1 0 0 1 0 1 0 1]
 [0 1 1 0 1 0 1 0]]
